In [35]:
from keras.models import load_model
from utils import CircularStack
import airsim
import cv2
import numpy as np
import time

In [36]:
model = load_model('model.h5df')

In [42]:
client = airsim.CarClient()
client.confirmConnection()


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [43]:
num_images_input = 4
cstack = CircularStack(num_images_input)
car_controls = airsim.CarControls()
client.enableApiControl(True)

i = 0
delta_time = 0.1
while i < 300:
    
    response = client.simGetImages(
        [airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])[0]
    img = np.frombuffer(response.image_data_uint8, dtype=np.uint8)
    img = img.reshape(response.height, response.width, 4)[..., :3]
    img = cv2.resize(img, (128, 128))
    
    cstack.update(img)
    
    if cstack.is_full():
        imgs_stack = np.concatenate([img for img in cstack], axis=2)[np.newaxis, ...]
        steering_angle = model.predict(imgs_stack)
        car_controls.throttle = 0.2
        car_controls.steering = float(steering_angle[0][0])

        client.setCarControls(car_controls)
    
    time.sleep(delta_time)
    i += 1
    
client.enableApiControl(False)